In [ ]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.animation import FuncAnimation
from tqdm import tqdm_notebook as tqdm
from matplotlib.patches import Patch

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathos.multiprocessing import ProcessingPool as Pool
import pandas as pd
import seaborn as sns
import random
from itertools import product
import plotly.graph_objects as go
from itertools import chain

from joblib import Parallel, delayed
import random
from joblib import dump


import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
import CPC_package as CPC
%matplotlib inline

import sys
import os

# Is is more symmetric or asymmetric to build bridges

In [16]:
def add_prefix(node, prefix):
    return f"{prefix}_{node}"

def plot_the_graphs(graph, pos):
    plt.figure(figsize=(10, 10))
    nx.draw(graph, pos, node_size=20)
    plt.show()


def checkFlowFromAtoB(graph):
    handler = CPC.CpcHandler(graph.copy(), cores=10, seed_function=CPC.randomFactorSeed)
    handler.setPortion(portion=0.05)
    handler.number_of_seeds = 10*graph.number_of_nodes()
    handler.setRandomFactor(0)
    handler.to_dict_representation()
    handler.setThresholds(2)
    
    #get the list of nodes in graph_A
    nodes_A = [node for node in graph.nodes() if node.startswith('A')]

    sd, activatedNodes = handler.makeSingleSpread(set(nodes_A))

    return sd, activatedNodes

def checkFlowFromBtoA(graph):
    handler = CPC.CpcHandler(graph.copy(), cores=10, seed_function=CPC.randomFactorSeed)
    handler.setPortion(portion=0.05)
    handler.number_of_seeds = 10*graph.number_of_nodes()
    handler.setRandomFactor(0)
    handler.to_dict_representation()
    handler.setThresholds(2)
    
    #get the list of nodes in graph_A
    nodes_B= [node for node in graph.nodes() if node.startswith('B')]

    sd, activatedNodes = handler.makeSingleSpread(set(nodes_B))

    return sd, activatedNodes

def addRandomTieBetweenGraphs(graph):
    #get the nodes of graph_A and graph_B
    nodes_A = [node for node in graph.nodes() if node.startswith('A')]
    nodes_B = [node for node in graph.nodes() if node.startswith('B')]

    success = False

    while not success:
        #pick a random node from graph_A and graph_B
        node_A = random.choice(nodes_A)
        node_B = random.choice(nodes_B)

        #check if there is already a tie between the two nodes
        if not graph.has_edge(node_A, node_B):
            success = True
            #add a tie between the two nodes
            graph.add_edge(node_A, node_B)

    return graph

def getGraph():
    N, k, beta = 100, 6, 0.1
    #create a WS graph
    graph_A = nx.watts_strogatz_graph(N, k, beta)
    graph_B = nx.watts_strogatz_graph(N, k, beta)
    #relabel all graphs in graph_A by adding the prefix 'A' to the node label
    graph_A = nx.relabel_nodes(graph_A, lambda node: add_prefix(node, 'A'))
    #relabel all graphs in graph_B by adding the prefix 'B' to the node label
    graph_B = nx.relabel_nodes(graph_B, lambda node: add_prefix(node, 'B'))

    #calculate the position of the nodes in the graph_A and graph_B
    pos_A = nx.kamada_kawai_layout(graph_A)
    pos_B = nx.kamada_kawai_layout(graph_B)
    #move the nodes of graph_B to the right
    for node in pos_B:
        pos_B[node][0] += 3

    #merge the positions of graph_A and graph_B
    pos = {}
    pos.update(pos_A)
    pos.update(pos_B)

    #merge the graphs into one graph
    graph = nx.compose(graph_A, graph_B)
    return graph, pos

def addTriadicClosureBetweenGraphs(graph):
    #get the nodes of graph_A and graph_B
    nodes_A = [node for node in graph.nodes() if node.startswith('A')]
    nodes_B = [node for node in graph.nodes() if node.startswith('B')]

    success = False

    while not success:
        #pick a random node from graph_A and graph_B
        node_A = random.choice(nodes_A)
        node_B = random.choice(nodes_B)

        #check if there is already a tie between the two nodes
        if graph.has_edge(node_A, node_B):
            if random.random() < 0.5:
                #make closure from A to neib of B
                #pick a random neighbor of node_B
                neighbor_B = random.choice(list(graph.neighbors(node_B)))
                #check if there is already a tie between node_A and neighbor_B
                if not graph.has_edge(node_A, neighbor_B) and node_A != neighbor_B:
                    done = True
                    #add a tie between the two nodes
                    graph.add_edge(node_A, neighbor_B)
                    success = True
            else:
                #make closure from B to neib of A
                #pick a random neighbor of node_A
                neighbor_A = random.choice(list(graph.neighbors(node_A)))
                #check if there is already a tie between node_B and neighbor_A
                if not graph.has_edge(node_B, neighbor_A) and node_B != neighbor_A:
                    #add a tie between the two nodes
                    graph.add_edge(node_B, neighbor_A)
                    success = True
            #add a tie between the two nodes
            graph.add_edge(node_A, node_B)

    return graph

# Bridge formation triadic closure

In [17]:
prob_for_triadic_closure = [0, 0.2, 0.4, 0.6, 0.8, 1.0]

In [ ]:
def run_simulation(p, i):
    random.seed(int(i))
    graph, pos = getGraph()
    sd_AtoB, activatedNodes_AtoB = checkFlowFromAtoB(graph)
    sd_BtoA, activatedNodes_BtoA = checkFlowFromBtoA(graph)
    local_results = []
    counter = 0
    while counter < 40:
        counter += 1
        if counter == 1 or random.random() >= p:
            graph = addRandomTieBetweenGraphs(graph)
        else:
            graph = addTriadicClosureBetweenGraphs(graph)
        sd_AtoB, activatedNodes_AtoB = checkFlowFromAtoB(graph)
        sd_BtoA, activatedNodes_BtoA = checkFlowFromBtoA(graph)

        if sd_AtoB < 1:
            sd_AtoB = 0
        if sd_BtoA < 1:
            sd_BtoA = 0

        local_results.append((p, i, counter, sd_AtoB, sd_BtoA, abs(sd_AtoB - sd_BtoA)))
    return local_results

results = []
for p in tqdm(prob_for_triadic_closure, desc="Probability for Triadic Closure"):
    parallel_results = Parallel(n_jobs=-1)(delayed(run_simulation)(p, i) for i in range(1000))
    for sublist in parallel_results:
        results.extend(sublist)

Probability for Triadic Closure: 100%|██████████| 6/6 [04:50<00:00, 48.50s/it]


In [ ]:
#create a df from the results
df = pd.DataFrame(results, columns=['p','iteration', 'counter', 'sd_AtoB', 'sd_BtoA', 'difference'])
#create a column from the sum of sd_AtoB and sd_BtoA
df['sum'] = df['sd_AtoB'] + df['sd_BtoA']
#what is the minimal counter where the sum is 2
min_counter = df[df['sum'] == 2]['counter'].min()
print(min_counter)

3


In [ ]:
#check if there is any iteration that jumped from 0 to 1 in one step and has no other values
df = df[(df['sd_AtoB'] == 1) | (df['sd_BtoA'] == 1)]
#do a value count grouped by counter and difference and create a new df
df_grouped = df.groupby(['counter', 'difference', 'p']).size().reset_index(name='counts')
#for each counter calculate the portion of counts where the difference is 0
df_grouped['portion'] = df_grouped.groupby(['counter','p'])['counts'].transform(lambda x: x / x.sum())
#drop all rows where difference is not 0
df_grouped = df_grouped[df_grouped['difference'] == 0]

In [21]:
dump(df_grouped, 'df_random_bridge_building_triadic.joblib')

['df_random_bridge_building_triadic.joblib']